In [6]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import re    
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Dropout
from tensorflow.keras.models import Model

In [7]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수:', len(lines))
lines.sample(5)

전체 샘플의 수: 197463


,eng,fra,cc
78957,I don't need that anymore.,Je n'en ai plus besoin.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
106147,Tom drank a lot more than me.,Tom a bu beaucoup plus que moi.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
172633,Why can't you take things just as they are?,Pourquoi ne peux-tu pas accepter les choses ju...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
63007,He got tired of reading.,Il commence à en avoir assez de lire.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
189419,Have you noticed a change in the size or color...,Avez-vous remarqué un changement de taille ou ...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [8]:
lines = lines[['eng', 'fra']][60000:93000]
lines.sample(5)

,eng,fra
92214,Who's making the decisions?,Qui prend les décisions ?
82515,The company is in the red.,L'entreprise est dans le rouge.
76215,Where exactly did you go?,"Où es-tu allée, exactement ?"
61985,A hyphen is needed here.,Il faut un trait d'union ici.
79045,I enjoyed the party a lot.,J'ai beaucoup apprécié la fête.


In [9]:
lines_np_eng= lines['eng'].to_numpy()
lines_np_fra= lines['fra'].to_numpy()
lines_np_eng

array(['Those are your choices.', 'Those are your options.',
       'Those are your options.', ..., 'Are you coming to the party?',
       'Are you coming to the party?', 'Are you done with the paper?'],
      dtype=object)

# 데이터 전처리하기

구두점 분리/ 소문자로 변경하기

In [10]:
sos_token = '<start> '
eos_token = ' <end>'

def preprocess_line(line, plus_token = True):
    
    line = line.lower().strip() #1
    line = re.sub(r"([?.!,¿])", r" \1 ", line) #2
    line = re.sub(r'[" "]+', " ", line) #3
    line = re.sub(r"[^a-zA-Z?.!,¿]+", " ", line) #4

    line = line.strip()
    
    if plus_token == True:
        line = sos_token + line + eos_token
    
    return line

*1 소문자로 바꾸고, 양쪽 공백을 지운다
*2 특수문자 양옆에 공백을 넣는다
*3 여러개의 공백은 하나의 공백으로 바꾼다
*4 a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다

띄어쓰기 단위로 tokenize

In [11]:
def tokenize(corpus):
    tokenizer = Tokenizer(
        num_words=7000,  # 7000 단어를 기억할 수 있는 토크나이저 만들기
        filters=' ',   # 이미 문장을 정제했으니 필터 노필요!
        oov_token="<unk>"  # 7000단어에 포함되지 못한 단어는 <unk>으로 바꿈
    )
    tokenizer.fit_on_texts(corpus)  # corpus를 이용해 토크나이저 내부의 단어장을 완성

    tensor = tokenizer.texts_to_sequences(corpus)   # 토크나이저를 이용해 corpus를 tensor로 변환

    return tensor, tokenizer

> tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드

# 영어랑 프랑스어 따로 전처리

In [26]:
eng_lines = []
fra_lines = []

# eng_lines.append(lines.eng.apply(lambda x : preprocess_line(x,plus_token = False)))
# fra_lines.append(lines.fra.apply(lambda x : preprocess_line(x),))

for eng, fra in zip(lines.eng, lines.fra):
    if len(eng) == 0: continue
    if len(fra) == 0: continue   
        
    eng_lines.append(preprocess_line(eng, plus_token = False))
    fra_lines.append(preprocess_line(fra))

In [27]:
np.shape(eng_lines)

(33000,)

In [14]:
eng_tensor, eng_tokenizer = tokenize(eng_lines)
fra_tensor, fra_tokenizer = tokenize(fra_lines)
fra_tensor[:10]

[[2, 1155, 175, 1274, 4, 3],
 [2, 1064, 22, 61, 42, 1274, 4, 3],
 [2, 21, 61, 175, 1274, 4, 3],
 [2, 21, 61, 42, 1274, 4, 3],
 [2, 178, 672, 36, 480, 98, 4, 3],
 [2, 178, 316, 27, 14, 61, 9, 44, 4, 3],
 [2, 33, 59, 30, 126, 2309, 378, 4, 3],
 [2, 178, 897, 61, 2646, 4, 3],
 [2, 17, 89, 8, 24, 3799, 316, 4, 3],
 [2, 33, 8, 499, 47, 44, 4, 3]]

# input & target 설정

In [15]:
encoder_input = eng_tensor
# 종료 토큰 제거
decoder_input = [[char for char in line if char != fra_tokenizer.word_index['<end>']] for line in fra_tensor]
# 시작 토큰 제거
decoder_target =[[char for char in line if char != fra_tokenizer.word_index['<start>']] for line in fra_tensor]

> 이때, 디코더의 입력으로 사용할 시퀀스는 < eos >토큰이 필요가 없고, 디코더의 출력과 비교할 시퀀스는 < sos >가 필요가 없기 때문입니다. 가령, 영어로 'I am a person'이라는 문장을 프랑스어 'Je suis une personne'로 번역하는 번역기를 만든다고 해봅시다. 훈련 과정에서 디코더는 '< sos > Je suis une personne'를 입력받아서 'Je suis une personne < eos >'를 예측하도록 훈련되므로, 이런 방식으로 생성된 두가지 버전의 시퀀스를 준비해야 합니다.

# padding

In [18]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 5975
프랑스어 단어장의 크기 : 8501


In [21]:
max_eng_seq_len = max([len(line) for line in eng_lines])
max_fra_seq_len = max([len(line) for line in fra_lines])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 31
프랑스어 시퀀스의 최대 길이 91


In [22]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 31)
프랑스어 입력데이터의 크기(shape) : (33000, 91)
프랑스어 출력데이터의 크기(shape) : (33000, 91)


In [11]:
def pad_tensor(tensor):
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = max(num_tokens)
#     max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    tensor = pad_sequences(tensor, padding='post', maxlen=maxlen)  
    return tensor

> 이것도 패딩을 뒤말고 앞으로 할 수 있나??

In [12]:
encoder_input = pad_tensor(encoder_input)
decoder_input = pad_tensor(decoder_input)
decoder_target = pad_tensor(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 10)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [14]:
eng_vocab_size = len(eng_tokenizer.word_index)+1
fra_vocab_size = len(fra_tokenizer.word_index)+1

max_eng_seq_len = encoder_input.shape[1]
max_fra_seq_len = decoder_input.shape[1]

In [15]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 5975
프랑스어 단어장의 크기 : 8501
영어 시퀀스의 최대 길이 10
프랑스어 시퀀스의 최대 길이 17


# Validation

In [16]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [17]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 10)
(30000, 17)
(30000, 17)
(3000, 10)
(3000, 17)
(3000, 17)


# Embedding layer 사용하기

인코더

In [18]:
embedding_size = 512
hidden_size = 512
# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None, ), name='encoder_input')
enc_emb =  Embedding(eng_vocab_size, embedding_size,
                    input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(hidden_size, dropout = 0.5, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

디코더

In [19]:
decoder_inputs = Input(shape=(None, ), name='decoder_input')
dec_emb =  Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(hidden_size, dropout = 0.5, return_sequences = True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

In [20]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [21]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 512)    3059200     encoder_input[0][0]              
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 512)    4352512     decoder_input[0][0]              
______________________________________________________________________________________________

In [23]:
model.fit(x=[encoder_input_train, decoder_input_train], 
          y=decoder_target_train, 
          validation_data = ([encoder_input_test, decoder_input_test], 
                             decoder_target_test),
          batch_size=32, 
          epochs=20)

Epoch 1/20
938/938 [==============================] - 49s 21ms/step - loss: 2.0039 - val_loss: 1.6603
Epoch 2/20
938/938 [==============================] - 17s 18ms/step - loss: 1.5168 - val_loss: 1.4231
Epoch 3/20
938/938 [==============================] - 17s 18ms/step - loss: 1.3157 - val_loss: 1.2837
Epoch 4/20
938/938 [==============================] - 17s 18ms/step - loss: 1.1754 - val_loss: 1.1921
Epoch 5/20
938/938 [==============================] - 17s 19ms/step - loss: 1.0692 - val_loss: 1.1252
Epoch 6/20
938/938 [==============================] - 18s 19ms/step - loss: 0.9805 - val_loss: 1.0843
Epoch 7/20
938/938 [==============================] - 18s 19ms/step - loss: 0.9125 - val_loss: 1.0521
Epoch 8/20
938/938 [==============================] - 18s 19ms/step - loss: 0.8597 - val_loss: 1.0298
Epoch 9/20
938/938 [==============================] - 17s 19ms/step - loss: 0.8193 - val_loss: 1.0286
Epoch 10/20
938/938 [==============================] - 18s 19ms/step - loss: 0.793

# 모델 구현하기

인코더

In [24]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 512)         3059200   
_________________________________________________________________
masking (Masking)            (None, None, 512)         0         
_________________________________________________________________
lstm (LSTM)                  [(None, 512), (None, 512) 2099200   
Total params: 5,158,400
Trainable params: 5,158,400
Non-trainable params: 0
_________________________________________________________________


디코더

In [25]:
decoder_state_input_h = Input(shape=(embedding_size,)) # 이전 timestep의 hidden state를 저장하는 텐서
decoder_state_input_c = Input(shape=(embedding_size,)) # 이전 타입스텝의 cell state를 저장하는 텐서
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c] # 이젠 타입스텝의 히든 스테이트와 셀스테이트를 하나의 변수에 저장

dec_emb2 = Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2) # 디코더의 출력층 설계

In [26]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [ ]:
> 단어에서 정수로, 정수에서 단어로 바꾸는 사전(dictionary)를 준비

In [27]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 512)    4352512     decoder_input[0][0]              
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512)]        0                                            
____________________________________________________________________________________________

In [28]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <start>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

> 예측과정을 위한 함수 decode_sequence()를 구현. decode_sequence()의 입력으로 들어가는 것은 번역하고자 하는 문장의 정수 시퀀스입니다. decode_sequence() 내부에는 인코더를 구현한 encoder_model이 있어서 이 모델에 번역하고자 하는 문장의 정수 시퀀스인 'input_seq'를 입력하면, encoder_model은 마지막 시점의 hidden state를 리턴합니다. 이 hidden state는 디코더의 첫번째 시점의 hidden state가 되고, 디코더는 이제 번역 문장을 완성하기 위한 예측 과정을 진행합니다. 디코더의 예측 과정에서는 이전 시점에서 예측한 단어를 디코더의 현재 시점의 입력으로 넣어주는 작업을 진행합니다. 그리고 이 작업은 종료를 의미하는 종료 토큰을 만나거나, 주어진 최대 길이를 넘을 때까지 반복합니다.

In [29]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [30]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

# 모델평가하기

In [31]:
for seq_index in [1,201,501,1004,2015]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

-----------------------------------
입력 문장: i really enjoyed myself . 
정답 문장: je me suis vraiment bien amus . 
번역기가 번역한 문장:  j me vraiment vraimen
-----------------------------------
입력 문장: the rent is due tomorrow . 
정답 문장: le loyer est d pour demain . 
번역기가 번역한 문장:  le est le demain demai
-----------------------------------
입력 문장: don t forget to wear a tie . 
정답 문장: n oubliez pas de porter une cravate . 
번역기가 번역한 문장:  n pas pas de d . 
-----------------------------------
입력 문장: everyone looked surprised . 
정답 문장: tout le monde a eu l air surpris . 
번역기가 번역한 문장:  tout le a a cela 
-----------------------------------
입력 문장: do you want that warmed up ? 
정답 문장: veux tu que je r chauffe cela ? 
번역기가 번역한 문장:  voulez vous que qu
